In [ ]:
import xarray as xr
import numpy as np

import geopandas as gp
import matplotlib.pyplot as plt

from nb_shared import *

from swe_compare.nb_helpers import start_cluster
from swe_compare import CBRFCZone, Snow17SWE, ZoneCompare
from swe_compare.rasterize_zone import \
    cbrfc_zone_mask_as_xr, single_cbrfc_zone_mask_as_xr

# Description

This notebook rasterizes a single CBRFC zone shape into a gridded raster based on the SWANN raster grid.

In [ ]:
upperColorado = CBRFCZone(UC_ZONES)
cbrfc_zones = cbrfc_zone_mask_as_xr(UC_ZONE_TIF, upperColorado)

In [ ]:
s17_swe = Snow17SWE(
    SWE_HOME_DIR + 'CBRFC_Gunnison_1981_2020_SWE_inches.csv',
)
target_zones = list(s17_swe.csv.columns)
target_zones = upperColorado.target_zones_as_dict(target_zones)
target_zones

In [ ]:
alec2_huf = upperColorado.shapefile.query("zone == 'ALEC2HUF'")

In [ ]:
alec2_huf.bounds

In [ ]:
swann = xr.open_mfdataset(
    SWANN_HOME_DIR + 'SWE_Mask_v01.nc',
)

In [ ]:
bottom_right = swann.sel(
    lat=alec2_huf.bounds.miny.values[0],
    lon=alec2_huf.bounds.minx.values[0],
    method="nearest"
)
top_left = swann.sel(
    lat=alec2_huf.bounds.maxy.values[0],
    lon=alec2_huf.bounds.maxx.values[0],
    method="nearest"
)

In [ ]:
top_left

In [ ]:
bottom_right

In [ ]:
lat_mean_diff = np.mean(np.diff(swann.lat.values)) / 2
print(lat_mean_diff)
lon_mean_diff = np.mean(np.diff(swann.lon.values)) / 2
print(lon_mean_diff)
lat_values = swann.lat.values - lat_mean_diff
# lat_values = np.insert(lat_values, 0, lat_values[-1] + lon_mean_diff)

In [ ]:
alec2_huf_tif = single_cbrfc_zone_mask_as_xr(
    UC_ZONE_TIF, alec2_huf
)

In [ ]:
alec2_huf_tif = alec2_huf_tif.sel(
    lat=slice(alec2_huf.bounds.miny.values[0], alec2_huf.bounds.maxy.values[0]),
    lon=slice(alec2_huf.bounds.minx.values[0], alec2_huf.bounds.maxx.values[0]),
)

In [ ]:
fig = plt.figure(dpi=150)
ax = fig.gca()
alec2_huf_tif.zone.plot(ax=ax)
alec2_huf.plot(ax=ax, color='orange')
ax.hlines(lat_values, -108, -106, color='black', lw=1, ls=':')
ax.vlines(swann.lon.values - lon_mean_diff, 38, 40, color='black', lw=1, ls=':')
ax.set_xlim(alec2_huf.bounds.minx.values[0], alec2_huf.bounds.maxx.values[0])
ax.set_ylim(alec2_huf.bounds.miny.values[0], alec2_huf.bounds.maxy.values[0])
ax.tick_params(axis='x', rotation=-25)
ax.set_title('SWANN grid');